In [ ]:
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-50')
#glove_vectors100 = gensim.downloader.load('glove-twitter-100')
#lexvec = gensim.models.Word2Vec.load_word2vec_format('path-to-vectors.txt', binary=False)


In [ ]:
import json
import numpy as np
import pandas as pd
import os
import csv
import spacy
import nltk
import re
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')
nlp = spacy.load("en_core_web_sm")

reviewMap = {}
i = 0
with open("drive/MyDrive/Data_Final", "r", encoding="UTF-8") as f:
  f = csv.reader(f)
  for line in f:
    if i == 0:
      i = 1
      continue
    review = line[7]
    id = line[2]
    if(id in reviewMap):
      reviewMap[id].append(review)
    else:
      reviewMap[id] = [review]

print(len(reviewMap))

# clean words
def process(document):
  stopWords = nlp.Defaults.stop_words
  # Remove all the special characters, like parathesis
  document = re.sub(r'\W', ' ', document)
  # remove all single characters: like a, b, c, d
  document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
  # Remove single characters from the start
  document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
  # Substituting multiple spaces with single space
  document = re.sub(r'\s+', ' ', document, flags=re.I)
  # Removing prefixed 'b'
  document = re.sub(r'^b\s+', '', document)
  # Lower case
  document = document.lower()
  # Lemmatization
  document = document.split()
  document = [stemmer.lemmatize(word) for word in document]
  # Remove stop words
  document = [word for word in document if word not in stopWords]
  # Remove words that aren't in the embedding dictionary
  document = [word for word in document if word in glove_vectors]
  return document

print(list(reviewMap.values())[5])

for k in reviewMap.copy():
  textList = list(reviewMap[k])
  newList = []
  for text in textList:
    document = process(str(text))
    newList.append(document)
  reviewMap[k] = newList

print(list(reviewMap.values())[5])

nReviews = 0

# remove all reviews with less than 10 tokens
for k in reviewMap.copy():
  textList = list(reviewMap[k])
  newList = []
  for n in range(len(textList)):
    nReviews += 1
    if len(textList[n]) >= 10:
      newList.append(textList[n])
  reviewMap[k] = newList

print(len(reviewMap))

print('nReviews: ', nReviews)
nReviews = 0

# remove all businesses with less than 10 reviews 
for key, val in reviewMap.copy().items():
  nReviews += len(val)
  if len(val) < 10:
    del reviewMap[key]


print(len(reviewMap))

print('nReviews: ', nReviews)



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


3530
["Reviewing the food.\nHard to justify $16 for a burger and fries BUT wowow this burger was so delicious. I'm not sure if I was just starving but ordered the Lama burger and the patty size is quality. They also have random ingredients thrown in that you probably haven't seen before like gruyere stuffed in a pepper on your burger. Fries came out nice and crisp vs. soggy and the bun on that burger was great. You order inside by the Nook and then can sit around. They have tons of craft beer to choose from and I could see this place getting packed on the weekends.", 'Dog friendly and good selection of food and drinks. Cute outdoor patio. \n\nWe ordered the Nook Burger and Crab Cakes with slaw. \nBoth were delicious. I would come back again when in the area.', "I stopped by with my partner during happy hour and it was busy, but didn't seem to attract a rowdy crowd. There was a great mix of people of different ages and it felt very accessible. This is the first place I ever felt offered

Word embeddings

Gradient Boosting

Preparing for training

In [ ]:

lenArr = []
for arr in list(reviewMap.values()):
  for text in arr:
    lenArr.append(len(text))
print(np.min(lenArr))


keyMap = {}
xData = []
yData = []
i = 0
for key in reviewMap.keys():
  # save key
  keyMap[i] = key
  # embed
  for review in reviewMap[key]:
    # average all the words
    conglomerate = np.zeros(50)
    for word in review:
      conglomerate = np.add(conglomerate, glove_vectors[word])
    conglomerate = np.divide(conglomerate, np.ones_like(conglomerate)*len(review))
    yData.append(i)
    xData.append(conglomerate)
  i += 1


print(xData[1])



10
[ 1.36409922e-03 -4.49404899e-01  6.83623794e-02 -2.73458512e-02
  5.16417145e-01  8.04439330e-02  2.28551887e-02  8.19870463e-02
  7.83639039e-02 -3.72985694e-02 -4.88722376e-01 -1.27255237e-01
 -2.98615334e+00 -1.31051286e-01  7.87801434e-02  2.19984250e-01
  5.29778239e-01 -5.31794293e-01 -1.24499283e-01  3.49856157e-02
 -4.61840097e-01 -6.10586178e-02  9.99865048e-02 -4.85431378e-02
  8.97483320e-02  6.99964281e-01 -2.27834172e-02  3.18478070e-02
  3.65044969e-01 -4.87173498e-01  2.32351141e-01  6.43943092e-01
 -3.20626052e-01 -2.85771378e-01 -3.80955282e-04 -1.08000636e-02
  1.97740719e-01 -6.46640935e-02  4.93524472e-01  5.38814764e-01
 -5.56741565e-01 -2.18271999e-01  2.05051909e-01 -2.46532096e-01
  6.84470757e-01 -7.45712874e-02  5.08933197e-01 -6.66361923e-02
 -2.92279496e-02  1.60866333e-01]


In [ ]:
import math
allIndex = list(range(len(xData)))
testIndex = np.random.choice(allIndex, size=math.floor(len(xData)/10), replace=False)

mask = np.ones(len(xData), dtype=bool)
mask[testIndex] = False

xData = np.array(xData)
yData = np.array(yData)
xTrain = xData[mask]
yTrain = yData[mask]
xTest = xData[testIndex]
yTest = yData[testIndex]

In [ ]:
# Get n class
print(len(np.unique(yData)))

1036


In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(xTrain, label=yTrain)
dtest = xgb.DMatrix(xTest, label=yTest)

param = {'max_depth': 6, 'eta': 0.15, 'objective': 'multi:softprob', 'num_class': 1036}
param['nthread'] = 4
param['eval_metric'] = 'mlogloss'
evallist = [(dtrain, 'train'), (dtest, 'eval')]

num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	train-mlogloss:6.0913	eval-mlogloss:6.50577
[1]	train-mlogloss:5.46966	eval-mlogloss:6.2315
[2]	train-mlogloss:4.9908	eval-mlogloss:6.05641
[3]	train-mlogloss:4.58176	eval-mlogloss:5.92554
[4]	train-mlogloss:4.22446	eval-mlogloss:5.83198
[5]	train-mlogloss:3.90447	eval-mlogloss:5.74311
[6]	train-mlogloss:3.6172	eval-mlogloss:5.67172
[7]	train-mlogloss:3.35906	eval-mlogloss:5.60692
[8]	train-mlogloss:3.1225	eval-mlogloss:5.55538
[9]	train-mlogloss:2.90529	eval-mlogloss:5.50899


In [ ]:
print(bst.best_iteration)
bst.save_model('101C-02.model')

9


In [ ]:

import xgboost as xgb
xgb.__version__

'0.90'

In [ ]:
import json
json.dump(keyMap, open('keyMap.json', 'w'))